# Carregar pacotes

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Ajustando configuração para lidar com saídas longas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('expand_frame_repr', False)

# Desabilitando o truncamento: 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Carregar o conjunto de dados

In [2]:
# Definindo o caminho relativo dos arquivos 
data_path = os.getcwd()  # Obtém o diretório atual do notebook

# Carregando os conjuntos de dados
student_mat = pd.read_csv(os.path.join(data_path, "student-mat.csv"), sep=";")
student_por = pd.read_csv(os.path.join(data_path, "student-por.csv"), sep=";")
obesity_file = os.path.join(data_path, "ObesityDataSet_raw_and_data_sinthetic.csv")
obesity_data = pd.read_csv(obesity_file)  

# Exibindo os primeiros registros dos conjuntos de dados:
print("Dados sobre o desempenho dos alunos (Matemática):")
print(student_mat.head())
print("\nDados sobre o desempenho dos alunos (Português):")
print(student_por.head())
print("\nDados sobre a obesidade:")
print(obesity_data.head())

Dados sobre o desempenho dos alunos (Matemática):
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  reason guardian  traveltime  studytime  failures schoolsup famsup paid activities nursery higher internet romantic  famrel  freetime  goout  Dalc  Walc  health  absences  G1  G2  G3
0     GP   F   18       U     GT3       A     4     4  at_home   teacher  course   mother           2          2         0       yes     no   no         no     yes    yes       no       no       4         3      4     1     1       3         6   5   6   6
1     GP   F   17       U     GT3       T     1     1  at_home     other  course   father           1          2         0        no    yes   no         no      no    yes      yes       no       5         3      3     1     1       3         4   5   5   6
2     GP   F   15       U     LE3       T     1     1  at_home     other   other   mother           1          2         3       yes     no  yes         no     yes    yes      yes   

# Árvore de Decisão

In [3]:
def evaluate_decision_tree(X, y, dataset_name):
    print(f"\n--- Avaliação para o conjunto de dados: {dataset_name} ---")
    
    # Divisão em treino e teste
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    
    # Treinamento com Árvore de Decisão
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X_train, y_train)
    
    # Previsão nos conjuntos de treino e teste
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    
    # Avaliação: erros e desempenho
    train_accuracy = accuracy_score(y_train, y_train_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    print(f"Erro no conjunto de treinamento: {1 - train_accuracy:.4f}")
    print(f"Erro no conjunto de teste: {1 - test_accuracy:.4f}")
    print(f"Desempenho no conjunto de treinamento (Acurácia): {train_accuracy:.4f}")
    print(f"Desempenho no conjunto de teste (Acurácia): {test_accuracy:.4f}")
    
    # Validação cruzada
    cv_scores = cross_val_score(clf, X, y, cv=5)
    print(f"\nValidação Cruzada (média): {cv_scores.mean():.4f}")
    print(f"Relatório de Classificação no Teste:\n{classification_report(y_test, y_test_pred)}")

# Conjunto 1: Dados sobre obesidade

In [4]:
obesity_file = os.path.join(data_path, "ObesityDataSet_raw_and_data_sinthetic.csv")
obesity_data = pd.read_csv(obesity_file)
obesity_data = obesity_data.dropna()  # Removendo valores nulos

# Codificando a variável alvo 'NObeyesdad'
label_encoder = LabelEncoder()
y_obesity = label_encoder.fit_transform(obesity_data["NObeyesdad"])

# Selecionando apenas colunas numéricas
X_obesity = obesity_data.select_dtypes(include=[np.number])

# Conjunto 2: Desempenho estudantil

In [5]:
X_student = student_mat.drop("G1", axis=1).select_dtypes(include=[np.number])

# Criando faixas para a variável alvo 'G1'
bins = [0, 10, 15, 20] 
labels = ["Baixo", "Médio", "Alto"]
y_student = pd.cut(student_mat["G1"], bins=bins, labels=labels)

# Avaliação dos dois conjuntos

In [6]:
evaluate_decision_tree(X_obesity, y_obesity, "Obesidade")


--- Avaliação para o conjunto de dados: Obesidade ---
Erro no conjunto de treinamento: 0.0000
Erro no conjunto de teste: 0.0678
Desempenho no conjunto de treinamento (Acurácia): 1.0000
Desempenho no conjunto de teste (Acurácia): 0.9322

Validação Cruzada (média): 0.9323
Relatório de Classificação no Teste:
              precision    recall  f1-score   support

           0       0.94      0.90      0.92        82
           1       0.86      0.86      0.86        86
           2       0.90      0.98      0.94       106
           3       0.96      0.96      0.96        89
           4       1.00      0.97      0.98        97
           5       0.90      0.94      0.92        87
           6       0.97      0.90      0.93        87

    accuracy                           0.93       634
   macro avg       0.93      0.93      0.93       634
weighted avg       0.93      0.93      0.93       634



In [7]:
evaluate_decision_tree(X_student, y_student, "Desempenho Estudantil")


--- Avaliação para o conjunto de dados: Desempenho Estudantil ---
Erro no conjunto de treinamento: 0.0000
Erro no conjunto de teste: 0.2773
Desempenho no conjunto de treinamento (Acurácia): 1.0000
Desempenho no conjunto de teste (Acurácia): 0.7227

Validação Cruzada (média): 0.7367
Relatório de Classificação no Teste:
              precision    recall  f1-score   support

        Alto       0.70      0.58      0.64        12
       Baixo       0.80      0.74      0.77        58
       Médio       0.65      0.73      0.69        49

    accuracy                           0.72       119
   macro avg       0.72      0.69      0.70       119
weighted avg       0.73      0.72      0.72       119

